In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))  # or "../../" depending on location

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
# from models.ResNet18 import get_resnet18_for_cifar10
from models.CNN import CNN

import os
from tqdm import tqdm

In [ ]:
# data load
# 前処理（ToTensor + 正規化）
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.261))
])

# ダウンロード済みのデータを指定して読み込み
train_dataset = datasets.CIFAR10(
    root="../data/cifar10", 
    train=True, 
    download=False,  
    transform=transform_train
)

test_dataset = datasets.CIFAR10(
    root="../data/cifar10", 
    train=False, 
    download=False, 
    transform=transform_test
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

subset_size = 10000
indices = np.random.choice(len(train_dataset), subset_size, replace=False)

# サブセットデータセットを作成
small_train_dataset = Subset(train_dataset, indices)
train_loader_small = DataLoader(small_train_dataset, batch_size=128, shuffle=True)


In [4]:
# train models
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")
model = CNN()
model = model.to(device)

Using device: mps


In [5]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader_small):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}")

    # 検証
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    acc = 100. * correct / total
    print(f"Test Accuracy: {acc:.2f}%")


100%|██████████| 79/79 [00:02<00:00, 33.68it/s]


[Epoch 1] Train Loss: 0.4017
Test Accuracy: 39.04%


100%|██████████| 79/79 [00:01<00:00, 41.09it/s]


[Epoch 2] Train Loss: 0.3407
Test Accuracy: 46.19%


100%|██████████| 79/79 [00:01<00:00, 40.98it/s]


...

[Epoch 38] Train Loss: 0.1192
Test Accuracy: 72.96%


100%|██████████| 79/79 [00:01<00:00, 41.34it/s]


[Epoch 39] Train Loss: 0.1130
Test Accuracy: 72.73%


100%|██████████| 79/79 [00:01<00:00, 39.66it/s]


[Epoch 40] Train Loss: 0.1094
Test Accuracy: 73.60%


In [ ]:
torch.save(model.state_dict(), "../models/pretrained/CNN_cifar10_small.pth")